In [38]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import nltk

from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.utils import to_categorical

In [2]:

with open('data/train.txt', 'r') as file:
    train = file.readlines()
    
with open('data/test.txt', 'r') as file:
    test = file.readlines()

In [21]:
df_train = pd.DataFrame(train, columns=['text'])
df_train[['text', 'emotion']] = df_train['text'].str.split(';', expand=True)
 
df_test = pd.DataFrame(test, columns=['text'])
df_test[['text', 'emotion']] = df_test['text'].str.split(';', expand=True)

In [24]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [26]:
nltk.download('wordnet')
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df_train['processed_text'] = df_train['text'].apply(preprocess_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/karpagapriyadhanraj/nltk_data...


In [27]:
label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['emotion'])
df_test['label'] = label_encoder.fit_transform(df_test['emotion'])
num_classes = len(label_encoder.classes_)
Y = to_categorical(df_train['label'], num_classes=num_classes)


/Users/karpagapriyadhanraj/miniconda3/envs/my-env/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/karpagapriyadhanraj/miniconda3/envs/my-env/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/karpagapriyadhanraj/miniconda3/envs/my-env/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/karpagapriyadhanraj/miniconda3/envs/my-env/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and 

In [36]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
oov_tok = "<OOV>"


tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(df_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df_train['text'])
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
print(len(padded))

test_sequence = tokenizer.texts_to_sequences(df_test)
testing_padded = pad_sequences(test_sequence,maxlen=max_length)

16000


In [29]:
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(32)),
    Dense(6, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 bidirectional (Bidirection  (None, 64)                12544     
 al)                                                             
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 172941 (675.55 KB)
Trainable params: 172941 (675.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
num_epochs = 50
history = model.fit(padded, df_train, epochs=num_epochs, validation_data=(testing_padded, df_test))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).